In [0]:
from pyspark.sql.types import *

tick_schema = StructType([
  StructField("ts", TimestampType(), True),
  StructField("open", DoubleType(), True),
  StructField("high", DoubleType(), True),
  StructField("low", DoubleType(), True),
  StructField("price", DoubleType(), True),
  StructField("volume", IntegerType(), True),
  StructField("symbol", StringType(), True)
])

In [0]:
tick_df = spark.read.csv("/FileStore/all_stocks_5yr.csv",
                         header = True,
                         schema = tick_schema)

In [0]:
tick_df.count()

In [0]:
tick_df = tick_df.drop("open", "high", "low", "volume")

In [0]:
tick_df = tick_df.sort("ts", "symbol")

In [0]:
tick_df.show(10)

In [0]:
%run ./Setup

In [0]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [0]:
(tick_df.write
   .format("singlestore")
   .option("loadDataCompression", "LZ4")
   .mode("ignore")
   .save("timeseries_db.tick"))